In [131]:
# Load neccessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
import seaborn as sns
import matplotlib as mpl

In [132]:
# Load the training dataset
df_policies=pd.read_csv("auto_policies_2020.csv")
df_policies
#df_policies.isnull().sum() 

,pol_number,pol_eff_dt,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,claim_office,numclaims,claimcst0,annual_premium
0,43124327,12/30/2020,F,4.0,7/12/1968,381.0,D,133.6,2,HBACK,1.331,NaN,0,0.0,716.53
1,21919609,12/30/2020,F,2.0,11/5/1982,549.0,D,163.6,1,UTE,3.740,NaN,0,0.0,716.53
2,72577057,12/30/2020,M,2.0,11/26/1983,649.0,B,117.5,4,COUPE,0.880,NaN,0,0.0,716.53
3,92175225,12/30/2020,M,4.0,11/2/1960,743.0,B,100.7,3,SEDAN,1.045,NaN,0,0.0,716.53
4,66223239,12/30/2020,F,4.0,1/4/1968,817.0,C,115.5,4,HBACK,0.473,NaN,0,0.0,716.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60387,73248694,1/2/2020,F,5.0,5/10/1956,809.0,C,145.5,4,HBACK,0.891,NaN,0,0.0,716.53
60388,71411764,1/2/2020,M,4.0,3/22/1961,681.0,D,86.3,1,HBACK,1.881,NaN,0,0.0,716.53
60389,89353155,1/2/2020,M,4.0,9/29/1965,773.0,F,110.0,1,STNWG,5.170,NaN,0,0.0,716.53
60390,40916605,1/2/2020,M,3.0,8/1/1978,714.0,B,NaN,1,HBACK,1.903,NaN,0,0.0,716.53


In [133]:
# Fill the traffic index column using the average mean of the column
df_policies['traffic_index'] = df_policies['traffic_index'].fillna(df_policies['traffic_index'].mean())

# Dropping rows with missing values
df_policies.dropna(axis=0,how='any',subset=['credit_score'],inplace=True)
df_policies.isnull().sum()
df_policies

,pol_number,pol_eff_dt,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,claim_office,numclaims,claimcst0,annual_premium
0,43124327,12/30/2020,F,4.0,7/12/1968,381.0,D,133.600000,2,HBACK,1.331,NaN,0,0.0,716.53
1,21919609,12/30/2020,F,2.0,11/5/1982,549.0,D,163.600000,1,UTE,3.740,NaN,0,0.0,716.53
2,72577057,12/30/2020,M,2.0,11/26/1983,649.0,B,117.500000,4,COUPE,0.880,NaN,0,0.0,716.53
3,92175225,12/30/2020,M,4.0,11/2/1960,743.0,B,100.700000,3,SEDAN,1.045,NaN,0,0.0,716.53
4,66223239,12/30/2020,F,4.0,1/4/1968,817.0,C,115.500000,4,HBACK,0.473,NaN,0,0.0,716.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60386,47830088,1/2/2020,F,3.0,8/2/1977,755.0,E,34.500000,4,UTE,2.310,NaN,0,0.0,716.53
60387,73248694,1/2/2020,F,5.0,5/10/1956,809.0,C,145.500000,4,HBACK,0.891,NaN,0,0.0,716.53
60388,71411764,1/2/2020,M,4.0,3/22/1961,681.0,D,86.300000,1,HBACK,1.881,NaN,0,0.0,716.53
60389,89353155,1/2/2020,M,4.0,9/29/1965,773.0,F,110.000000,1,STNWG,5.170,NaN,0,0.0,716.53


In [134]:
# Formating the date column to be a readable data in python using .to_datetime() function
df_policies['date_of_birth']= pd.to_datetime(df_policies['date_of_birth'],format='%m/%d/%Y')
# Created columns based on column using date time package.
df_policies['year']= df_policies['date_of_birth'].dt.year
df_policies['month']= df_policies['date_of_birth'].dt.month
df_policies['day']= df_policies['date_of_birth'].dt.day
df_policies

,pol_number,pol_eff_dt,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,claim_office,numclaims,claimcst0,annual_premium,year,month,day
0,43124327,12/30/2020,F,4.0,1968-07-12,381.0,D,133.600000,2,HBACK,1.331,NaN,0,0.0,716.53,1968,7,12
1,21919609,12/30/2020,F,2.0,1982-11-05,549.0,D,163.600000,1,UTE,3.740,NaN,0,0.0,716.53,1982,11,5
2,72577057,12/30/2020,M,2.0,1983-11-26,649.0,B,117.500000,4,COUPE,0.880,NaN,0,0.0,716.53,1983,11,26
3,92175225,12/30/2020,M,4.0,1960-11-02,743.0,B,100.700000,3,SEDAN,1.045,NaN,0,0.0,716.53,1960,11,2
4,66223239,12/30/2020,F,4.0,1968-01-04,817.0,C,115.500000,4,HBACK,0.473,NaN,0,0.0,716.53,1968,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60386,47830088,1/2/2020,F,3.0,1977-08-02,755.0,E,34.500000,4,UTE,2.310,NaN,0,0.0,716.53,1977,8,2
60387,73248694,1/2/2020,F,5.0,1956-05-10,809.0,C,145.500000,4,HBACK,0.891,NaN,0,0.0,716.53,1956,5,10
60388,71411764,1/2/2020,M,4.0,1961-03-22,681.0,D,86.300000,1,HBACK,1.881,NaN,0,0.0,716.53,1961,3,22
60389,89353155,1/2/2020,M,4.0,1965-09-29,773.0,F,110.000000,1,STNWG,5.170,NaN,0,0.0,716.53,1965,9,29


In [135]:
# Binned values for training data
binned_values_train = []

# Creating the bins using an if-else statement
for value in df_policies['year']:
    if value >= 1990:
        temp = "1.0"
    elif value >= 1980 and value <= 1989:
        temp = "2.0"
    elif value >= 1970 and value <= 1979:
        temp = "3.0"
    elif value >= 1960 and value <= 1969:
        temp = "4.0"
    elif value >= 1950 and value <= 1959:
        temp = "5.0"
    else:
        temp = "6.0"
    binned_values_train.append(temp)

In [136]:
# Creating the new column in the training dataset
df_policies["New_agecat"] = binned_values_train
df_policies

,pol_number,pol_eff_dt,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,claim_office,numclaims,claimcst0,annual_premium,year,month,day,New_agecat
0,43124327,12/30/2020,F,4.0,1968-07-12,381.0,D,133.600000,2,HBACK,1.331,NaN,0,0.0,716.53,1968,7,12,4.0
1,21919609,12/30/2020,F,2.0,1982-11-05,549.0,D,163.600000,1,UTE,3.740,NaN,0,0.0,716.53,1982,11,5,2.0
2,72577057,12/30/2020,M,2.0,1983-11-26,649.0,B,117.500000,4,COUPE,0.880,NaN,0,0.0,716.53,1983,11,26,2.0
3,92175225,12/30/2020,M,4.0,1960-11-02,743.0,B,100.700000,3,SEDAN,1.045,NaN,0,0.0,716.53,1960,11,2,4.0
4,66223239,12/30/2020,F,4.0,1968-01-04,817.0,C,115.500000,4,HBACK,0.473,NaN,0,0.0,716.53,1968,1,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60386,47830088,1/2/2020,F,3.0,1977-08-02,755.0,E,34.500000,4,UTE,2.310,NaN,0,0.0,716.53,1977,8,2,3.0
60387,73248694,1/2/2020,F,5.0,1956-05-10,809.0,C,145.500000,4,HBACK,0.891,NaN,0,0.0,716.53,1956,5,10,5.0
60388,71411764,1/2/2020,M,4.0,1961-03-22,681.0,D,86.300000,1,HBACK,1.881,NaN,0,0.0,716.53,1961,3,22,4.0
60389,89353155,1/2/2020,M,4.0,1965-09-29,773.0,F,110.000000,1,STNWG,5.170,NaN,0,0.0,716.53,1965,9,29,4.0


In [137]:
# Label encoder
le = preprocessing.LabelEncoder()
df_policies['gender'] = le.fit_transform(df_policies['gender'])
df_policies['area'] = le.fit_transform(df_policies['area'])
# Creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(encoder.fit_transform(df_policies[["veh_body"]]).toarray())
enc_df.columns = encoder.get_feature_names_out(["veh_body"])
# Merge with main df on the row index - essentially pressing the datasets together 
df_policies = pd.concat([df_policies.reset_index(drop=True), enc_df.reset_index(drop=True)], axis = 1)
df_policies

,pol_number,pol_eff_dt,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,...,veh_body_HBACK,veh_body_HDTOP,veh_body_MCARA,veh_body_MIBUS,veh_body_PANVN,veh_body_RDSTR,veh_body_SEDAN,veh_body_STNWG,veh_body_TRUCK,veh_body_UTE
0,43124327,12/30/2020,0,4.0,1968-07-12,381.0,3,133.600000,2,HBACK,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21919609,12/30/2020,0,2.0,1982-11-05,549.0,3,163.600000,1,UTE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,72577057,12/30/2020,1,2.0,1983-11-26,649.0,1,117.500000,4,COUPE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,92175225,12/30/2020,1,4.0,1960-11-02,743.0,1,100.700000,3,SEDAN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,66223239,12/30/2020,0,4.0,1968-01-04,817.0,2,115.500000,4,HBACK,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57586,47830088,1/2/2020,0,3.0,1977-08-02,755.0,4,34.500000,4,UTE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
57587,73248694,1/2/2020,0,5.0,1956-05-10,809.0,2,145.500000,4,HBACK,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57588,71411764,1/2/2020,1,4.0,1961-03-22,681.0,3,86.300000,1,HBACK,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57589,89353155,1/2/2020,1,4.0,1965-09-29,773.0,5,110.000000,1,STNWG,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [138]:
# Dropping unnecessary columns
policies_updated=df_policies.drop(['pol_number','agecat','pol_eff_dt','month','day','date_of_birth','veh_body','annual_premium','claim_office'],axis=1)
policies_updated

,gender,credit_score,area,traffic_index,veh_age,veh_value,numclaims,claimcst0,year,New_agecat,...,veh_body_HBACK,veh_body_HDTOP,veh_body_MCARA,veh_body_MIBUS,veh_body_PANVN,veh_body_RDSTR,veh_body_SEDAN,veh_body_STNWG,veh_body_TRUCK,veh_body_UTE
0,0,381.0,3,133.600000,2,1.331,0,0.0,1968,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,549.0,3,163.600000,1,3.740,0,0.0,1982,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,649.0,1,117.500000,4,0.880,0,0.0,1983,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,743.0,1,100.700000,3,1.045,0,0.0,1960,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,817.0,2,115.500000,4,0.473,0,0.0,1968,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57586,0,755.0,4,34.500000,4,2.310,0,0.0,1977,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
57587,0,809.0,2,145.500000,4,0.891,0,0.0,1956,5.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57588,1,681.0,3,86.300000,1,1.881,0,0.0,1961,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57589,1,773.0,5,110.000000,1,5.170,0,0.0,1965,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [139]:
policies_updated['numclaims'].value_counts()

0    48245
1     7487
2     1244
3      430
4      138
5       47
Name: numclaims, dtype: int64

In [167]:
# Create features that are strings and target
x = policies_updated

# Create features that are strings and target
x_train = policies_updated.drop(['numclaims','claimcst0'], axis = 1)
y_train1 = policies_updated.numclaims
y_train2 = policies_updated.claimcst0

from imblearn.over_sampling import SMOTE

# Define SMOTE function with a fixed random state of 42
sm = SMOTE(random_state=42, k_neighbors = 5)

# New X and Y dataset
x_bal, y_bal = sm.fit_resample(x_train, y_train1)
y_bal.value_counts()

0    48245
1    48245
2    48245
3    48245
4    48245
5    48245
Name: numclaims, dtype: int64

In [168]:
# Feature selection using a variance threshold
from sklearn.feature_selection import VarianceThreshold

# Define variance threshold
sel = VarianceThreshold(threshold = 0.95)
# Fit transformation
sel.fit_transform(x_bal)
# Get select features
sel.get_support()

array([False,  True,  True,  True, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [116]:
#Obtaining the new train set based opn the selected features
X_selected = x_bal.iloc[:, [1, 2, 3, 5, 6, 7]]
X_selected

,credit_score,area,traffic_index,veh_value,year,New_agecat
0,381.000000,3,133.600000,1.331000,1968,4.0
1,549.000000,3,163.600000,3.740000,1982,2.0
2,649.000000,1,117.500000,0.880000,1983,2.0
3,743.000000,1,100.700000,1.045000,1960,4.0
4,817.000000,2,115.500000,0.473000,1968,4.0
...,...,...,...,...,...,...
289465,801.822641,0,105.656826,1.931358,1948,6.0
289466,555.394532,3,41.602862,3.214192,1944,5.821615
289467,387.640905,1,92.204147,1.387583,1974,2.820453
289468,351.966038,1,135.991341,1.017680,1979,2.528103


In [117]:
#load the testing data
df_potential=pd.read_csv("auto_potential_customers_2022.csv")
df_potential.isnull().sum()
#df_potential

quote_number       0
gender             0
agecat           632
date_of_birth      0
credit_score     355
area               0
traffic_index    449
veh_age            0
veh_body           0
veh_value          0
dtype: int64

In [118]:
# Fill the traffic index column using the average mean of the column
df_potential['traffic_index'] = df_potential['traffic_index'].fillna(df_potential['traffic_index'].mean())

#dropping rows with missing values
df_potential.dropna(axis=0,how='any',subset=['credit_score'],inplace=True)

In [120]:
# Formating the date column to be a readable data in python using .to_datetime() function
df_potential['date_of_birth']= pd.to_datetime(df_potential['date_of_birth'],format='%m/%d/%Y')
# Created columns based on column using date time package.
df_potential['year']= df_potential['date_of_birth'].dt.year
df_potential['month']= df_potential['date_of_birth'].dt.month
df_potential['day']= df_potential['date_of_birth'].dt.day
df_potential

,quote_number,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,year,month,day
0,29323463,F,4.0,1968-04-14,750.0,C,124.500000,2,HBACK,1.100,1968,4,14
1,16732441,M,2.0,1984-10-04,567.0,F,104.416636,3,STNWG,2.079,1984,10,4
2,68744228,M,4.0,1962-11-30,378.0,D,111.400000,3,HDTOP,2.354,1962,11,30
3,30875047,F,3.0,1973-05-17,807.0,E,104.416636,4,UTE,1.969,1973,5,17
4,37049210,M,6.0,1945-10-02,844.0,C,111.000000,1,HBACK,1.969,1945,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7459,95907219,F,4.0,1966-01-07,690.0,D,71.400000,4,STNWG,1.056,1966,1,7
7460,73906733,M,5.0,1958-05-25,558.0,C,142.500000,1,STNWG,4.521,1958,5,25
7461,45407372,F,4.0,1966-09-02,527.0,A,62.700000,1,STNWG,3.102,1966,9,2
7462,41423165,M,5.0,1958-08-09,817.0,B,129.000000,4,SEDAN,0.803,1958,8,9


In [121]:
# Binned values for training data
binned_values_train = []

# Creating the bins using an if-else statement
for value in df_potential['year']:
    if value >= 1990:
        temp = "1.0"
    elif value >= 1980 and value <= 1989:
        temp = "2.0"
    elif value >= 1970 and value <= 1979:
        temp = "3.0"
    elif value >= 1960 and value <= 1969:
        temp = "4.0"
    elif value >= 1950 and value <= 1959:
        temp = "5.0"
    else:
        temp = "6.0"
    binned_values_train.append(temp)

In [124]:
# Creating the new columns in my training and testing dataset
df_potential["New_agecat"] = binned_values_train
df_potential

,quote_number,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,year,month,day,new_agecat,New_agecat
0,29323463,F,4.0,1968-04-14,750.0,C,124.500000,2,HBACK,1.100,1968,4,14,4.0,4.0
1,16732441,M,2.0,1984-10-04,567.0,F,104.416636,3,STNWG,2.079,1984,10,4,2.0,2.0
2,68744228,M,4.0,1962-11-30,378.0,D,111.400000,3,HDTOP,2.354,1962,11,30,4.0,4.0
3,30875047,F,3.0,1973-05-17,807.0,E,104.416636,4,UTE,1.969,1973,5,17,3.0,3.0
4,37049210,M,6.0,1945-10-02,844.0,C,111.000000,1,HBACK,1.969,1945,10,2,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7459,95907219,F,4.0,1966-01-07,690.0,D,71.400000,4,STNWG,1.056,1966,1,7,4.0,4.0
7460,73906733,M,5.0,1958-05-25,558.0,C,142.500000,1,STNWG,4.521,1958,5,25,5.0,5.0
7461,45407372,F,4.0,1966-09-02,527.0,A,62.700000,1,STNWG,3.102,1966,9,2,4.0,4.0
7462,41423165,M,5.0,1958-08-09,817.0,B,129.000000,4,SEDAN,0.803,1958,8,9,5.0,5.0


In [125]:
# Label encoder
le = preprocessing.LabelEncoder()
df_potential['gender'] = le.fit_transform(df_potential['gender'])
df_potential['area'] = le.fit_transform(df_potential['area'])
# creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(encoder.fit_transform(df_potential[["veh_body"]]).toarray())
enc_df.columns = encoder.get_feature_names_out(["veh_body"])
# merge with main df on the row index - essentially pressing the datasets together 
df_potential = pd.concat([df_potential.reset_index(drop=True), enc_df.reset_index(drop=True)], axis = 1)
df_potential

,quote_number,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,...,veh_body_HBACK,veh_body_HDTOP,veh_body_MCARA,veh_body_MIBUS,veh_body_PANVN,veh_body_RDSTR,veh_body_SEDAN,veh_body_STNWG,veh_body_TRUCK,veh_body_UTE
0,29323463,0,4.0,1968-04-14,750.0,2,124.500000,2,HBACK,1.100,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16732441,1,2.0,1984-10-04,567.0,5,104.416636,3,STNWG,2.079,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,68744228,1,4.0,1962-11-30,378.0,3,111.400000,3,HDTOP,2.354,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30875047,0,3.0,1973-05-17,807.0,4,104.416636,4,UTE,1.969,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,37049210,1,6.0,1945-10-02,844.0,2,111.000000,1,HBACK,1.969,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7104,95907219,0,4.0,1966-01-07,690.0,3,71.400000,4,STNWG,1.056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7105,73906733,1,5.0,1958-05-25,558.0,2,142.500000,1,STNWG,4.521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7106,45407372,0,4.0,1966-09-02,527.0,0,62.700000,1,STNWG,3.102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7107,41423165,1,5.0,1958-08-09,817.0,1,129.000000,4,SEDAN,0.803,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [128]:
#dropping unnecessary columns
potential_updated = df_potential.drop(['quote_number','month','date_of_birth','agecat','day','veh_body'],axis=1)
potential_updated

,gender,credit_score,area,traffic_index,veh_age,veh_value,year,new_agecat,New_agecat,veh_body_BUS,...,veh_body_HBACK,veh_body_HDTOP,veh_body_MCARA,veh_body_MIBUS,veh_body_PANVN,veh_body_RDSTR,veh_body_SEDAN,veh_body_STNWG,veh_body_TRUCK,veh_body_UTE
0,0,750.0,2,124.500000,2,1.100,1968,4.0,4.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,567.0,5,104.416636,3,2.079,1984,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,378.0,3,111.400000,3,2.354,1962,4.0,4.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,807.0,4,104.416636,4,1.969,1973,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,844.0,2,111.000000,1,1.969,1945,6.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7104,0,690.0,3,71.400000,4,1.056,1966,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7105,1,558.0,2,142.500000,1,4.521,1958,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7106,0,527.0,0,62.700000,1,3.102,1966,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7107,1,817.0,1,129.000000,4,0.803,1958,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [129]:
x_test = potential_updated
x_test

,gender,credit_score,area,traffic_index,veh_age,veh_value,year,new_agecat,New_agecat,veh_body_BUS,...,veh_body_HBACK,veh_body_HDTOP,veh_body_MCARA,veh_body_MIBUS,veh_body_PANVN,veh_body_RDSTR,veh_body_SEDAN,veh_body_STNWG,veh_body_TRUCK,veh_body_UTE
0,0,750.0,2,124.500000,2,1.100,1968,4.0,4.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,567.0,5,104.416636,3,2.079,1984,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,378.0,3,111.400000,3,2.354,1962,4.0,4.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,807.0,4,104.416636,4,1.969,1973,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,844.0,2,111.000000,1,1.969,1945,6.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7104,0,690.0,3,71.400000,4,1.056,1966,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7105,1,558.0,2,142.500000,1,4.521,1958,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7106,0,527.0,0,62.700000,1,3.102,1966,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7107,1,817.0,1,129.000000,4,0.803,1958,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [157]:
#Selecting the same columns as in the training set (x_selected)

new_x_test=x_test[['credit_score','area','traffic_index','veh_value','year','New_agecat']]
new_x_test               

,credit_score,area,traffic_index,veh_value,year,New_agecat
0,750.0,2,124.500000,1.100,1968,4.0
1,567.0,5,104.416636,2.079,1984,2.0
2,378.0,3,111.400000,2.354,1962,4.0
3,807.0,4,104.416636,1.969,1973,3.0
4,844.0,2,111.000000,1.969,1945,6.0
...,...,...,...,...,...,...
7104,690.0,3,71.400000,1.056,1966,4.0
7105,558.0,2,142.500000,4.521,1958,5.0
7106,527.0,0,62.700000,3.102,1966,4.0
7107,817.0,1,129.000000,0.803,1958,5.0


In [ ]:
#Decision Tree

In [156]:
# prepare the cross-validation procedure for predicting numclaims
cv = KFold(n_splits=20, random_state=4, shuffle=True)

# Initialize decision tree regressor
dt_reg = DecisionTreeRegressor()

# Fit and score
dt_reg = dt_reg.fit(X_selected,y_bal)

# get the "scores" i.e. error for each fold
scores = cross_val_score(dt_reg, X_selected, y_bal, scoring='f1_weighted', cv=cv)

# Take the average of the scores to get an estimate of the model error
np.mean(scores)

0.899546676789963

In [158]:
predictions = dt_reg.predict(new_x_test)
new_x_test['numclaims']= predictions # Create column of estimated numclaims
new_x_test

C:\Users\david\AppData\Local\Temp/ipykernel_1400/1599897368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_x_test['numclaims']= predictions # Create column of estimated numclaims


,credit_score,area,traffic_index,veh_value,year,New_agecat,numclaims
0,750.0,2,124.500000,1.100,1968,4.0,0.0
1,567.0,5,104.416636,2.079,1984,2.0,0.0
2,378.0,3,111.400000,2.354,1962,4.0,1.0
3,807.0,4,104.416636,1.969,1973,3.0,1.0
4,844.0,2,111.000000,1.969,1945,6.0,0.0
...,...,...,...,...,...,...,...
7104,690.0,3,71.400000,1.056,1966,4.0,0.0
7105,558.0,2,142.500000,4.521,1958,5.0,0.0
7106,527.0,0,62.700000,3.102,1966,4.0,0.0
7107,817.0,1,129.000000,0.803,1958,5.0,0.0


In [172]:
# prepare the cross-validation procedure for predicting claimcst0
cv = KFold(n_splits=20, random_state=3, shuffle=True)

# Initialize decision tree regressor
dt_reg2 = DecisionTreeRegressor()

# Fit and score
dt_reg2 = dt_reg2.fit(x_train,y_train2)

# get the "scores" i.e. error for each fold
scores2 = cross_val_score(dt_reg2, x_train, y_train2, scoring='neg_mean_absolute_error', cv=cv)

# Take the average of the scores to get an estimate of the model error
np.mean(scores2)

-921.7627819182035